# Exploración de Datos con DuckDB

Este notebook demuestra cómo usar DuckDB para exploración interactiva de datos mediante SQL.

**Ventajas de DuckDB:**
- 10-100x más rápido que Pandas para queries analíticos
- Interfaz SQL familiar para exploración de datos
- Lee Parquet directamente sin cargar en memoria
- Base de datos portable en un solo archivo

**Autor:** Data Engineering Team  
**Fecha:** 2025-10-16

In [1]:
# Imports
import sys

sys.path.append('../..')

# Usar funciones reutilizables de utils
import plotly.express as px

from src.data.load_to_duckdb import get_table_info
from src.utils.duckdb_utils import (
    get_correlation,
    get_stats_by_column,
    get_temporal_stats,
    get_top_n,
    get_weekend_vs_weekday_stats,
    quick_query,
    setup_database,  # Nueva función para notebooks
)

## 1. Configuración de la Base de Datos

In [2]:
# Configurar base de datos (conecta y carga datos automáticamente si es necesario)
conn = setup_database(
    db_path="../../data/steel.duckdb",
    parquet_path="../../data/processed/steel_cleaned.parquet"
)

# Esta función:
# 1. Conecta a la base de datos
# 2. Verifica si los datos ya están cargados
# 3. Si no están, los carga automáticamente
# 4. Si ya están, solo informa y continúa

INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.utils.duckdb_utils:✅ Tabla 'steel_cleaned' ya existe en la base de datos


## 2. Información de la Tabla

In [3]:
# Obtener información de la tabla
info = get_table_info(conn, "steel_cleaned")

print(f"Tabla: {info['table_name']}")
print(f"Filas: {info['row_count']:,}")
print(f"Columnas: {info['column_count']}")
print("\nEsquema:")
for col_name, col_type in info['schema']:
    print(f"  {col_name}: {col_type}")

Tabla: steel_cleaned
Filas: 34,933
Columnas: 11

Esquema:
  date: VARCHAR
  Usage_kWh: DOUBLE
  Lagging_Current_Reactive.Power_kVarh: DOUBLE
  Leading_Current_Reactive_Power_kVarh: DOUBLE
  CO2(tCO2): DOUBLE
  Lagging_Current_Power_Factor: DOUBLE
  Leading_Current_Power_Factor: DOUBLE
  NSM: BIGINT
  WeekStatus: VARCHAR
  Day_of_week: VARCHAR
  Load_Type: VARCHAR


## 3. Queries Básicos

Usando funciones reutilizables de `src/utils/duckdb_utils.py` para queries comunes.

In [4]:
# Query 1: Primeras 10 filas usando quick_query (no necesita manejar conexión)
df = quick_query("SELECT * FROM steel_cleaned LIMIT 10", db_path="../../data/steel.duckdb")
df

INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 10 rows returned


date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
str,f64,f64,f64,f64,f64,f64,i64,str,str,str
"""01/01/2018 00:15""",3.17,2.95,0.0,0.0,73.21,100.0,900,"""Weekday""","""Monday""","""Light_Load"""
"""01/01/2018 00:30""",4.0,4.46,0.0,0.0,66.77,100.0,1800,"""Weekday""","""Monday""","""Light_Load"""
"""01/01/2018 00:45""",3.24,3.28,0.0,0.0,70.28,100.0,2700,"""Weekday""","""Monday""","""Light_Load"""
"""01/01/2018 01:00""",3.31,3.56,0.0,0.0,68.09,100.0,3600,"""Weekday""","""Monday""","""Light_Load"""
"""01/01/2018 01:15""",3.82,4.5,0.0,0.0,64.72,100.0,4500,"""Weekday""","""Monday""","""Light_Load"""
"""01/01/2018 01:30""",3.28,3.56,0.0,0.0,67.76,100.0,5400,"""Weekday""","""Monday""","""Light_Load"""
"""01/01/2018 01:45""",3.6,4.14,0.0,0.0,65.62,100.0,6300,"""Weekday""","""Monday""","""Light_Load"""
"""01/01/2018 02:00""",3.6,4.28,0.0,0.0,64.37,100.0,7200,"""Weekday""","""Monday""","""Light_Load"""
"""01/01/2018 02:15""",3.28,3.64,0.0,0.0,66.94,100.0,8100,"""Weekday""","""Monday""","""Light_Load"""


In [5]:
# Query 2: Estadísticas por Load_Type usando función reutilizable
df_stats = get_stats_by_column(
    table_name='steel_cleaned',
    group_by_column='Load_Type',
    agg_column='Usage_kWh',
    db_path="../../data/steel.duckdb"
)

df_stats

INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 4 rows returned


Load_Type,count,avg,min,max,std
str,i64,f64,f64,f64,f64
"""Maximum_Load""",7164,60.38,2.92,149.65,31.06
"""Medium_Load""",9584,39.83,2.59,149.65,37.02
"""Nan""",19,35.33,2.81,130.03,40.72
"""Light_Load""",18166,10.31,2.59,149.65,22.78


In [6]:
# Visualizar
fig = px.bar(
    df_stats.to_pandas(),
    x='Load_Type',
    y='avg',
    title='Consumo Energético Promedio por Tipo de Carga',
    labels={'avg': 'Uso Promedio (kWh)', 'Load_Type': 'Tipo de Carga'}
)
fig.show()

## 4. Análisis Temporal

Usando `get_temporal_stats()` para análisis por hora/día.

In [7]:
# Query 3: Consumo por Día de la Semana usando función reutilizable
df_day = get_stats_by_column(
    table_name='steel_cleaned',
    group_by_column='Day_of_week',
    agg_column='Usage_kWh',
    db_path="../../data/steel.duckdb"
)

df_day

INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 8 rows returned


Day_of_week,count,avg,min,max,std
str,i64,f64,f64,f64,f64
"""Thursday""",4922,36.39,2.59,149.65,37.98
"""Tuesday""",4926,35.52,2.59,149.65,37.52
"""Friday""",4946,34.72,2.59,149.65,37.06
"""Monday""",5332,34.2,2.59,149.65,37.21
"""Wednesday""",4939,33.43,2.59,149.65,37.33
"""Nan""",15,23.25,2.95,110.74,34.43
"""Saturday""",4933,17.25,2.59,149.65,26.68
"""Sunday""",4920,8.84,2.59,149.65,19.39


In [8]:
# Visualizar
fig = px.line(
    df_day.to_pandas(),
    x='Day_of_week',
    y='avg',
    title='Consumo Energético Promedio por Día de la Semana',
    markers=True
)
fig.show()

In [9]:
# Query 4: Análisis por Hora usando función reutilizable
df_hour = get_temporal_stats(
    table_name='steel_cleaned',
    time_column='NSM',
    agg_column='Usage_kWh',
    time_unit='hour',
    db_path="../../data/steel.duckdb"
)

df_hour.head(24)

INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 25 rows returned


hour,records,avg_Usage_kWh,std_Usage_kWh
f64,i64,f64,f64
0.0,1444,8.98,20.33
1.0,1449,8.0,20.3
2.0,1451,5.78,14.92
3.0,1449,5.53,14.26
4.0,1456,6.1,16.72
…,…,…,…
19.0,1454,39.49,35.99
20.0,1443,38.18,35.81
21.0,1453,15.49,25.01


In [10]:
# Visualizar patrón horario
fig = px.line(
    df_hour.to_pandas(),
    x='hour',
    y='avg_Usage_kWh',
    title='Consumo Energético Promedio por Hora del Día',
    markers=True
)
fig.update_xaxes(title='Hora del Día (0-23)')
fig.update_yaxes(title='Uso Promedio (kWh)')
fig.show()

## 5. Queries Avanzados

Usando funciones especializadas para análisis más complejos.

In [11]:
# Query 5: Top 10 Picos de Consumo usando función reutilizable
df_peaks = get_top_n(
    table_name='steel_cleaned',
    order_by_column='Usage_kWh',
    n=10,
    columns=['Usage_kWh', 'Load_Type', 'Day_of_week', 'NSM', 'CO2(tCO2)'],
    db_path="../../data/steel.duckdb"
)

df_peaks

INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 10 rows returned


Usage_kWh,Load_Type,Day_of_week,NSM,CO2(tCO2)
f64,str,str,i64,f64
149.65,"""Medium_Load""","""Tuesday""",52200,0.0
149.65,"""Light_Load""","""Wednesday""",22500,0.0
149.65,"""Medium_Load""","""Wednesday""",48600,0.02
149.65,"""Medium_Load""","""Friday""",49500,0.05
149.65,"""Light_Load""","""Saturday""",56700,0.0
149.65,"""Light_Load""","""Saturday""",63000,0.0
149.65,"""Maximum_Load""","""Tuesday""",65700,0.04
149.65,"""Light_Load""","""Friday""",22500,0.0
149.65,"""Medium_Load""","""Friday""",36000,0.05


In [12]:
# Query 6: Análisis de Correlación usando función reutilizable
print("Correlaciones con Usage_kWh:")
print(f"  CO2: {get_correlation('steel_cleaned', 'Usage_kWh', 'CO2(tCO2)', '../../data/steel.duckdb'):.4f}")
print(f"  Lagging Power Factor: {get_correlation('steel_cleaned', 'Usage_kWh', 'Lagging_Current_Power_Factor', '../../data/steel.duckdb'):.4f}")
print(f"  Leading Power Factor: {get_correlation('steel_cleaned', 'Usage_kWh', 'Leading_Current_Power_Factor', '../../data/steel.duckdb'):.4f}")

INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 1 rows returned
INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 1 rows returned
INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 1 rows returned


Correlaciones con Usage_kWh:
  CO2: 0.8941
  Lagging Power Factor: 0.3577
  Leading Power Factor: 0.3282


In [13]:
# Query 7: Análisis Fin de Semana vs Días Laborales usando función reutilizable
df_weekend = get_weekend_vs_weekday_stats(
    table_name='steel_cleaned',
    agg_column='Usage_kWh',
    db_path="../../data/steel.duckdb"
)

df_weekend

INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 3 rows returned


WeekStatus,records,avg,std,min,max
str,i64,f64,f64,f64,f64
"""Nan""",22,32.08,38.34,2.74,134.89
"""Weekday""",25069,34.84,37.43,2.59,149.65
"""Weekend""",9842,13.03,23.67,2.59,149.65


## 6. Análisis Personalizado

Usa esta celda para escribir tus propios queries SQL usando `quick_query()`:

In [14]:
# Tu query personalizado aquí usando quick_query
custom_query = """
    SELECT
        Load_Type,
        Day_of_week,
        COUNT(*) as count,
        ROUND(AVG(Usage_kWh), 2) as avg_usage
    FROM steel_cleaned
    GROUP BY Load_Type, Day_of_week
    ORDER BY Load_Type, Day_of_week
"""

df_custom = quick_query(custom_query, db_path="../../data/steel.duckdb")
df_custom

INFO:src.data.load_to_duckdb:✅ Connected to DuckDB: ../../data/steel.duckdb
INFO:src.data.load_to_duckdb:✅ Query executed: 31 rows returned


Load_Type,Day_of_week,count,avg_usage
str,str,i64,f64
"""Light_Load""","""Friday""",2200,13.36
"""Light_Load""","""Monday""",2446,10.47
"""Light_Load""","""Nan""",8,3.41
"""Light_Load""","""Saturday""",2830,8.38
"""Light_Load""","""Sunday""",3991,5.47
…,…,…,…
"""Nan""","""Saturday""",4,27.18
"""Nan""","""Sunday""",1,5.0
"""Nan""","""Thursday""",4,12.21


## 7. Cerrar Conexión

In [15]:
# Siempre cerrar la conexión al terminar
conn.close()
print("✅ Conexión cerrada")

✅ Conexión cerrada


## Resumen

Este notebook demostró:
1. ✅ Conexión a base de datos DuckDB usando funciones reutilizables
2. ✅ Carga de datos desde archivos Parquet
3. ✅ Ejecución de queries SQL con funciones de `src/utils/duckdb_utils.py`
4. ✅ Conversión de resultados a Polars DataFrames
5. ✅ Visualización de resultados de queries
6. ✅ Queries analíticos avanzados

**Funciones Reutilizables Usadas:**
- `get_connection()` - Conexión a DB
- `quick_query()` - Query rápido sin manejar conexión
- `get_stats_by_column()` - Estadísticas por columna
- `get_temporal_stats()` - Análisis temporal
- `get_top_n()` - Top N registros
- `get_correlation()` - Correlaciones
- `get_weekend_vs_weekday_stats()` - Comparación fin de semana

**Próximos Pasos:**
- Usar DuckDB para queries de feature engineering
- Crear views para queries comunes
- Exportar resultados para entrenamiento de modelos